In [33]:
##Swap Pricer
import numpy as np
from itertools import product
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
#Terms:
#Fixed Leg: semiannual pmts, 30/360 daycount, NYC Holidays
#Float Leg: quarterly pmts, ACT/360 daycount, 3ML w 2 day lookback,NYC & Lon Holidays
#Discount w the OIS Curve

In [56]:
def bdayconvert(start,end,dates):
    i = 0
    while i in range(len(dates)):
        if dates[i] not in pd.bdate_range(start,end):
            dates[i] = dates[i]+relativedelta(days=+1)
            i = 0
        else:
            i+=1
    return dates

In [116]:
def pmtdates(start,end):
    sdate = datetime.datetime.strptime(start, '%Y-%m-%d')
    edate = datetime.datetime.strptime(end, '%Y-%m-%d')
    delta = edate-sdate
    fixedpmtdates = [sdate+relativedelta(months=+6)]
    nyears = int(delta.days/365)
    i = 1
    while i < nyears*2:
        fixedpmtdates.append(fixedpmtdates[i-1]+relativedelta(months=+6))
        i+=1
    floatpmtdates = [sdate+relativedelta(months=+3)]
    i = 1
    while i < nyears*4:
        floatpmtdates.append(floatpmtdates[i-1]+relativedelta(months=+3))
        i+=1
    fixedpmtdates = bdayconvert(start,end,fixedpmtdates) 
    floatpmtdates = bdayconvert(start,end,floatpmtdates)
    return fixedpmtdates, floatpmtdates

In [ ]:
###add in later
#def hdayconvert(start,end,dates):
#    nycholidays = []
#    lonholidays = []
#    i = 0
#    while i in range(len(dates)):
#        if dates[i] not in pd.bdate_range(start,end):
#            dates[i] = dates[i]+relativedelta(days=+1)
#            i = 0
#        else:
#            i+=1
#    return dates

In [114]:
start = "2020-10-29"
end   = "2030-10-29"
notional = 100000000 #100 million
path1 = 'C:\\Users\\ECEI\\My Documents\\testdata.csv'
path2 = 'C:\\Users\\ECEI\\My Documents\\testois.csv'

In [123]:
start = input()
end = input()
notional = input()
path1 = input()
path2 = input()

2020-10-29
2030-10-29
100000000
C:\\Users\\ECEI\\My Documents\\testdata.csv
C:\\Users\\ECEI\\My Documents\\testois.csv


In [124]:
fixedpmtdates = pmtdates(start,end)[0]
floatpmtdates = pmtdates(start,end)[1]

In [120]:
ldata = pd.read_csv(path1)
odata = pd.read_csv(path2)

In [121]:
#getting the days for payment period
ldata['Dates'] = (floatpmtdates[0]-sdate).days
i = 1
while i < len(ldata['Dates']):
    ldata['Dates'][i] = (floatpmtdates[i]-floatpmtdates[i-1]).days
    i+=1
#calculating interest
ldata['Pmts'] = (notional*ldata['Forward Rate']/100)*ldata['Dates']/360
#calculating the zero
odata['Zerodf'] = (1/(1+odata['Forward Rate']/100))
#calculating discount factors
odata['DF'] = odata['Zerodf']
i=1
while i < len(odata['DF']):
    odata['DF'][i] = odata['Zerodf'][i]*odata['DF'][i-1]
    i+=1
#pv floatingpmts
ldata["PVpmts"]=1
for i in range(len(ldata["PVpmts"])):
    ldata["PVpmts"][i] = ldata["Pmts"][i]*odata["DF"][i]

C:\Users\ECEI\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\ECEI\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\ECEI\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [122]:
#calculating the mid
sum(ldata["PVpmts"])/sum(notional*(180/360)*odata['DF'])

0.005458445446352506